In [25]:
# 日本語フォントのインストール
%pip install japanize-matplotlib

Note: you may need to restart the kernel to use updated packages.


In [26]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import japanize_matplotlib

In [27]:
# 評価
class Evaluator:
    # 初期化
    def __init__(self, config):
        self.config = config
        self.model = None

    # データの前処理
    def preprocess_data(self, dataset):
        data = dataset.iloc[:, list(range(4, 12)) + [29] + list(range(19, 27))]
        label = dataset[['x_2', 'y_2']]
        return data.values, label.values

    # モデルのロード
    def load_model(self):
        self.model = keras.models.load_model(self.config['model_path'])

    # 累積正答率の計算
    def calc_accuracy(self, distance_err):
        # 許容誤差
        tolerances = np.arange(0, 2.1, 0.1).round(1)

        # 累積正答率
        correct_rates = [np.mean(distance_err <= tolerance) * 100 for tolerance in tolerances]
        
        # データの書き込み
        df = pd.DataFrame({'tolerances': tolerances, self.config['tole_name']: correct_rates})
        
        os.makedirs(os.path.dirname(self.config['tole_path']), exist_ok=True)
        
        if os.path.exists(self.config['tole_path']):
            existing_df = pd.read_csv(self.config['tole_path'])
            df = pd.merge(existing_df, df, on=['tolerances'], how='outer')
        
        df.to_csv(self.config['tole_path'], index=False)

    # ヒートマップの作成
    def make_heatmap(self, x_true, y_true, distance_err):
        # 閾値以下の正答率を計算
        tolerance = self.config['threshold']
        is_correct = (distance_err <= tolerance).astype(int)
        
        # 座標ごとに正答率の平均を計算
        heatmap_data = pd.DataFrame({'x': x_true, 'y': y_true, 'correct': is_correct})
        heatmap = heatmap_data.groupby(['x', 'y'])['correct'].mean().unstack()

        # プロット
        plt.figure(figsize=(9, 3))
        plt.imshow(heatmap.T, origin='lower', aspect='equal', cmap=plt.cm.Blues, interpolation='nearest', vmin=0, vmax=1)
        plt.colorbar(label='正答率')
        plt.xticks(np.arange(9) + 0.5, labels=np.arange(9))
        plt.yticks(np.arange(3) + 0.5, labels=np.arange(3))
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title('2時点の協調位置指紋')
        os.makedirs(os.path.dirname(self.config['heatmap_path']), exist_ok=True)
        plt.savefig(self.config['heatmap_path'])
        plt.close()

    # 評価
    def evaluate(self, test_dataset):
        # テストデータの前処理
        data, label = self.preprocess_data(test_dataset)

        # モデルのロード
        self.load_model()

        # 予測
        predict = self.model.predict(data)
        x_pred, y_pred = predict[:, 0], predict[:, 1]
        x_true, y_true = label[:, 0], label[:, 1]

        # 誤差の計算
        distance_err = np.sqrt((x_true - x_pred)**2 + (y_true - y_pred)**2)

        # 累積正答率、ヒートマップの作成
        self.calc_accuracy(distance_err)
        self.make_heatmap(x_true, y_true, distance_err)

In [28]:
# 設定
config = {
    'model_path': R'C:/Users/sukegawa/Desktop/study/model',
    'tole_path': R'C:/Users/sukegawa/Desktop/study/results/tolerances/test.csv',
    'tole_name': 'test',
    'heatmap_path': R'C:/Users/sukegawa/Desktop/study/results/heatmap/test.png'
}

# テストデータの評価
evaluator = Evaluator(config)
test_dataset = pd.read_csv(R'C:/Users/sukegawa/Desktop/study/datasets/tinvfp/tinvfp_test0.csv')
evaluator.evaluate(test_dataset)

345/345 [==============================] - 2s 4ms/step
